In [1]:
#Import Libraries
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sgt import SGT
import os 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import math
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler

from keras.optimizers import Adam, SGD, RMSprop
from keras.metrics import binary_accuracy, Precision, Recall, AUC

from sklearn.metrics import confusion_matrix
import seaborn as sns


2023-03-18 01:38:59.186104: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-18 01:38:59.679833: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-18 01:38:59.679880: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-18 01:38:59.679884: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
#Read Corpus
path13 = '../TOP_CON/Healthy/Healthy(1,3)/'
path5 = '../TOP_CON/Healthy/Healthy(5)/'
path6 = '../TOP_CON/Healthy/Healthy(6)/'

path4 = '../TOP_CON/Healthy/Healthy(4)/'
path7 = '../TOP_CON/Healthy/Healthy(7)/'
path9 = '../TOP_CON/Healthy/Healthy(9)/'
path12 = '../TOP_CON/Healthy/Healthy(12)/'


path13_g = '../TOP_CON/Glaucoma/Glaucoma(1,3)/'
path5_g = '../TOP_CON/Glaucoma/Glaucoma(5)/'
path6_g = '../TOP_CON/Glaucoma/Glaucoma(6)/'

path4_g = '../TOP_CON/Glaucoma/Glaucoma(4)/'
path7_g = '../TOP_CON/Glaucoma/Glaucoma(7)/'
path9_g = '../TOP_CON/Glaucoma/Glaucoma(9)/'
path12_g = '../TOP_CON/Glaucoma/Glaucoma(12)/'


paths = [path13,path5, path6, path4, path7, path9, path13_g,path5_g, path6_g, path4_g, path7_g, path9_g, path12, path12_g]
# folder_names = os.listdir(path)

def load_files(paths):
    l = []
    for folder in paths: 
        #p = path + folder + '/'
        p_os = os.listdir(folder)
        for filename in p_os:
            if filename!='.ipynb_checkpoints': 
                new_path = folder + filename 
                x = pd.read_csv(new_path)
                l.append(x)
    return l

corpus_novice = load_files(paths)

path8 = '../TOP_CON/Healthy/Healthy(8)/'
path8_g = '../TOP_CON/Glaucoma/Glaucoma(8)/'
path10 = '../TOP_CON/Healthy/Healthy(10)/'
path10_g = '../TOP_CON/Glaucoma/Glaucoma(10)/'
path11 = '../TOP_CON/Healthy/Healthy(11)/'
path11_g = '../TOP_CON/Glaucoma/Glaucoma(11)/'
path13 = '../TOP_CON/Healthy/Healthy(13)/'
path13_g = '../TOP_CON/Glaucoma/Glaucoma(13)/'
paths = [path8, path8_g, path10, path10_g, path11, path11_g, path13, path13_g]
# folder_names = os.listdir(path)

# path = '../TOP_CON/Glaucoma/'
# folder_names = os.listdir(path)

corpus_expert = load_files(paths)


for i in corpus_expert:
    i['fixation_id_new'] = i['fixation_id'] - i.iloc[0,2] + 1

for i in corpus_novice:
    i['fixation_id_new'] = i['fixation_id'] - i.iloc[0,2] + 1

corpus_expert

[    world_timestamp  world_index  fixation_id  start_timestamp  duration  \
 0       1189.575830          163           41      1189.401630  196.9820   
 1       1189.658194          164           42      1189.643630  163.0445   
 2       1189.746600          165           42      1189.643630  163.0445   
 3       1189.824831          166           42      1189.643630  163.0445   
 4       1190.082561          169           43      1190.079931   98.9240   
 5       1190.488646          174           44      1190.468340   98.2220   
 6       1190.591903          175           44      1190.468340   98.2220   
 7       1190.591903          175           45      1190.587578  147.4870   
 8       1190.694745          176           45      1190.587578  147.4870   
 9       1190.777443          177           46      1190.786160  215.2315   
 10      1190.871420          178           46      1190.786160  215.2315   
 11      1190.959132          179           46      1190.786160  215.2315   

In [3]:
def cat_encode(x,y):
    arr = []
    for i,j in zip(x,y):
        if 0<i and 0.6 > i and 0<j and 0.6 > j: 
            arr.append('A')
        elif 0<i and 0.25 > i and 0.6<j and 1 > j: 
            arr.append('B')
        elif 0.25<i and 0.48 > i and 0.6<j and 1 > j: 
            arr.append('C')
        elif 0.48<i and 0.6 > i and 0.6<j and 1 > j: 
            arr.append('D')
        elif 0.6<i and 1 > i and 0<j and 0.6 > j: 
            arr.append('E')
        elif 0.6<i and 0.77 > i and 0.6<j and 1 > j: 
            arr.append('F')
        elif 0.77<i and 1 > i and 0.6<j and 1 > j: 
            arr.append('G')
        else: 
            arr.append('error')

    return arr
    
def make_corpus(corpus_new):
    A = np.array(corpus_new.fixation_id_new[corpus_new.letters=='A'])
    B = np.array(corpus_new.fixation_id_new[corpus_new.letters=='B'])
    C = np.array(corpus_new.fixation_id_new[corpus_new.letters=='C'])
    D = np.array(corpus_new.fixation_id_new[corpus_new.letters=='D'])
    E = np.array(corpus_new.fixation_id_new[corpus_new.letters=='E'])
    F = np.array(corpus_new.fixation_id_new[corpus_new.letters=='F'])
    G = np.array(corpus_new.fixation_id_new[corpus_new.letters=='G'])
    corpus = pd.DataFrame([[1, A], 
                           [2, B],
                           [3, C], 
                           [4, D], 
                           [5, E], 
                           [6, F],
                           [7, G]], columns=['id', 'sequence'])
    return corpus

l = []
for i in corpus_novice: 
    arr = cat_encode(i.norm_pos_x, i.norm_pos_y)
    
    i['letters'] = arr
    i_new = i[['letters', 'fixation_id_new']]
    i_new = make_corpus(i_new)
    l.append(i_new)
    
for i in corpus_expert: 
    arr = cat_encode(i.norm_pos_x, i.norm_pos_y)
    i['letters'] = arr
    i_new = i[['letters', 'fixation_id_new']]
    i_new = make_corpus(i_new)
    l.append(i_new)
    


In [4]:
display(corpus_expert)

[    world_timestamp  world_index  fixation_id  start_timestamp  duration  \
 0       1189.575830          163           41      1189.401630  196.9820   
 1       1189.658194          164           42      1189.643630  163.0445   
 2       1189.746600          165           42      1189.643630  163.0445   
 3       1189.824831          166           42      1189.643630  163.0445   
 4       1190.082561          169           43      1190.079931   98.9240   
 5       1190.488646          174           44      1190.468340   98.2220   
 6       1190.591903          175           44      1190.468340   98.2220   
 7       1190.591903          175           45      1190.587578  147.4870   
 8       1190.694745          176           45      1190.587578  147.4870   
 9       1190.777443          177           46      1190.786160  215.2315   
 10      1190.871420          178           46      1190.786160  215.2315   
 11      1190.959132          179           46      1190.786160  215.2315   

In [5]:
e = []
for i in l: 
    sgt_ = SGT(kappa=1, 
               lengthsensitive=False, 
               mode='multiprocessing')
    sgtembedding_df = sgt_.fit_transform(i)
    sgtembedding_df = sgtembedding_df.set_index('id')
    e.append(sgtembedding_df)
    

print(len(e))
print(e[0])

INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

In [6]:
df2 = pd.DataFrame()
for i in e: 
    pca = PCA(n_components=1)
    pca.fit(i)
    X=pca.transform(i)
    df = pd.DataFrame(data=X, columns=['x1'])
    df_new = pd.DataFrame([[df.iloc[0,0], 
                   df.iloc[1,0], 
                   df.iloc[2,0], 
                  df.iloc[3,0],
                  df.iloc[4,0], 
                   df.iloc[5,0],
                  df.iloc[6,0]]], 
                 columns = ['0x1', '1x1', '2x1',  '3x1',  '4x1', '5x1',  '6x1'])
    df2 = pd.concat([df2, df_new])
df2 = df2.reset_index().drop(["index"], axis = 1)
labels = [1]*len(corpus_novice) + [0]*len(corpus_expert)

df2['labels'] = labels
df2

,0x1,1x1,2x1,3x1,4x1,5x1,6x1,labels
0,0.958814,-0.157562,-0.157562,-0.157562,-0.157562,-0.157562,-0.171003,1
1,0.492505,-0.076684,-0.076684,-0.076684,-0.076684,-0.109084,-0.076684,1
2,1.040909,-0.152488,-0.152488,-0.177155,-0.253801,-0.152488,-0.152488,1
3,0.583557,-0.083858,-0.083858,-0.083858,-0.119260,-0.106362,-0.106362,1
4,0.398856,-0.013757,-0.013757,-0.013757,-0.026890,-0.316938,-0.013757,1
...,...,...,...,...,...,...,...,...
234,-0.335446,-0.335446,-0.335446,-0.335446,2.027174,-0.335446,-0.349945,0
235,-0.145556,-0.145556,-0.145556,-0.145556,0.897793,-0.145556,-0.170011,0
236,-0.179408,-0.179408,-0.179408,-0.179408,1.149402,-0.179408,-0.252364,0
237,-0.167239,-0.167239,-0.167239,-0.167239,1.003436,-0.167239,-0.167239,0


In [7]:
df2['labels'].value_counts()

1    160
0     79
Name: labels, dtype: int64

In [8]:


labels = df2.labels
df_one_feature = df2.drop(['labels'], axis =1 )
col = df_one_feature.columns
mm = MinMaxScaler()
df_one_feature = mm.fit_transform(df_one_feature)
df_one_feature = df_one_feature*100
X_train, X_test, y_train, y_test = train_test_split(df_one_feature, labels, test_size=0.2, random_state=84)


In [42]:
def FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes):
    layers = []
    
    nodes_increment = (last_layer_nodes-first_layer_nodes)/ (n_layers-1)
    nodes = first_layer_nodes
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    return layers

FindLayerNodesLinear(4, 100, 30)

def createmodel(n_layers, first_layer_nodes, activation_func, loss_func, last_layer_nodes):
    model = Sequential()
    print(n_layers, first_layer_nodes, last_layer_nodes)
    n_nodes = FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes)
    for i in range(1, n_layers):
        if i==1:
            model.add(Dense(first_layer_nodes, input_dim=X_train.shape[1], activation=activation_func))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))
            
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='sgd', loss=loss_func, metrics = ["accuracy"]) 
    
    return model

model =  KerasClassifier(build_fn=createmodel, verbose = False)  

activation_funcs = ['relu', 'tanh'] 
loss_funcs = ['binary_crossentropy']
param_grid = dict(n_layers=np.arange(2,7), first_layer_nodes = [100], activation_func = activation_funcs, loss_func = loss_funcs, last_layer_nodes= [40,50,60], batch_size = [50], epochs = [25,50,75])
grid = GridSearchCV(estimator = model, param_grid = param_grid, n_jobs = -1)


/tmp/ipykernel_3886983/3634452454.py:28: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model =  KerasClassifier(build_fn=createmodel, verbose = False)


In [43]:
grid.fit(X_train,y_train)

2023-03-13 02:25:15.826283: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 02:25:15.826369: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 02:25:15.826521: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

2023-03-13 02:25:16.051220: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 02:25:16.073282: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 02:25:16.109504: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

2023-03-13 02:25:16.871532: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-13 02:25:16.871598: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-13 02:25:16.871606: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-13 02:25:16.881169: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory


2023-03-13 02:25:17.829783: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2023-03-13 02:25:17.829815: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-03-13 02:25:17.830062: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 02:25:17.831421: I

6 100 50


GridSearchCV(estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f3fac01a020>,
             n_jobs=-1,
             param_grid={'activation_func': ['relu', 'tanh'],
                         'batch_size': [50], 'epochs': [25, 50, 75],
                         'first_layer_nodes': [100],
                         'last_layer_nodes': [40, 50, 60],
                         'loss_func': ['binary_crossentropy'],
                         'n_layers': array([2, 3, 4, 5, 6])})

In [44]:
print(grid.best_score_)
print(grid.best_params_)

0.8105911374092102
{'activation_func': 'tanh', 'batch_size': 50, 'epochs': 75, 'first_layer_nodes': 100, 'last_layer_nodes': 50, 'loss_func': 'binary_crossentropy', 'n_layers': 6}


In [9]:

model_one_feature = Sequential()
model_one_feature.add(Dense(100, input_shape=(7,), activation='relu'))
model_one_feature.add(Dense(80, input_shape=(7,), activation='relu'))
model_one_feature.add(Dense(50, input_shape=(7,), activation='relu'))
# model_one_feature.add(Dense(50, input_shape=(7,), activation='relu'))

model_one_feature.add(Dense(1, activation='sigmoid'))




2023-03-18 01:40:52.924484: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-18 01:40:52.924700: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-18 01:40:52.950750: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2023-03-18 01:40:52.950773: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to do

In [53]:
optimizers = ['adam', 'sgd', 'rmsprop']
metrics = ['accuracy', Recall(), AUC()]
best_loss = float('inf')
best_optimizer = None
best_metric = None
# Compile the model with each optimizer and metric
for optimizer in optimizers:
    for metric in metrics:
        model_one_feature.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=metric)
        
        # Train and evaluate the model
        history = model_one_feature.fit(X_train, y_train, epochs=50, batch_size=50)
        val_loss, val_metric = model_one_feature.evaluate(X_test, y_test)
        
        # Print the validation loss and metric
#         print('Optimizer: {}, Metric: {}, Validation Loss: {:.4f}, Validation Metric: {:.4f}'.format(optimizer, metric, val_loss, val_metric))
        if val_loss < best_loss: 
            best_loss = val_loss
            best_optimizer = optimizer
            best_metric = metric
print(best_loss)
print(best_optimizer)
print(best_metric)
        







Epoch 1/50
3/3 [==============================] - 0s 2ms/step - loss: 1.0864 - accuracy: 0.7622
Epoch 2/50
3/3 [==============================] - 0s 2ms/step - loss: 0.7770 - accuracy: 0.6224
Epoch 3/50
3/3 [==============================] - 0s 2ms/step - loss: 0.6025 - accuracy: 0.7832
Epoch 4/50
3/3 [==============================] - 0s 2ms/step - loss: 0.5404 - accuracy: 0.7692
Epoch 5/50
3/3 [==============================] - 0s 2ms/step - loss: 0.4916 - accuracy: 0.7343
Epoch 6/50
3/3 [==============================] - 0s 1ms/step - loss: 0.4218 - accuracy: 0.7902
Epoch 7/50
3/3 [==============================] - 0s 1ms/step - loss: 0.4152 - accuracy: 0.7902
Epoch 8/50
3/3 [==============================] - 0s 1ms/step - loss: 0.4148 - accuracy: 0.7832
Epoch 9/50
3/3 [==============================] - 0s 1ms/step - loss: 0.4033 - accuracy: 0.7972
Epoch 10/50
3/3 [==============================] - 0s 1ms/step - loss: 0.3942 - accuracy: 0.7972
Epoch 11/50
3/3 [======================

3/3 [==============================] - 0s 1ms/step - loss: 0.2742 - accuracy: 0.8601
Epoch 19/50
3/3 [==============================] - 0s 2ms/step - loss: 0.2787 - accuracy: 0.8881
Epoch 20/50
3/3 [==============================] - 0s 2ms/step - loss: 0.2774 - accuracy: 0.8462
Epoch 21/50
3/3 [==============================] - 0s 2ms/step - loss: 0.2770 - accuracy: 0.8601
Epoch 22/50
3/3 [==============================] - 0s 1ms/step - loss: 0.3037 - accuracy: 0.8252
Epoch 23/50
3/3 [==============================] - 0s 1ms/step - loss: 0.2921 - accuracy: 0.8252
Epoch 24/50
3/3 [==============================] - 0s 1ms/step - loss: 0.2593 - accuracy: 0.8881
Epoch 25/50
3/3 [==============================] - 0s 1ms/step - loss: 0.2396 - accuracy: 0.8741
Epoch 26/50
3/3 [==============================] - 0s 1ms/step - loss: 0.3587 - accuracy: 0.7832
Epoch 27/50
3/3 [==============================] - 0s 2ms/step - loss: 0.4031 - accuracy: 0.8042
Epoch 28/50
3/3 [=========================

3/3 [==============================] - 0s 1ms/step - loss: 0.1788 - accuracy: 0.9021
Epoch 22/50
3/3 [==============================] - 0s 1ms/step - loss: 0.1656 - accuracy: 0.9301
Epoch 23/50
3/3 [==============================] - 0s 2ms/step - loss: 0.2232 - accuracy: 0.9091
Epoch 24/50
3/3 [==============================] - 0s 1ms/step - loss: 0.3969 - accuracy: 0.7832
Epoch 25/50
3/3 [==============================] - 0s 1ms/step - loss: 0.1765 - accuracy: 0.9371
Epoch 26/50
3/3 [==============================] - 0s 1ms/step - loss: 0.1700 - accuracy: 0.9371
Epoch 27/50
3/3 [==============================] - 0s 1ms/step - loss: 0.1696 - accuracy: 0.9161
Epoch 28/50
3/3 [==============================] - 0s 2ms/step - loss: 0.1743 - accuracy: 0.9161
Epoch 29/50
3/3 [==============================] - 0s 2ms/step - loss: 0.1572 - accuracy: 0.9441
Epoch 30/50
3/3 [==============================] - 0s 2ms/step - loss: 0.1516 - accuracy: 0.9371
Epoch 31/50
3/3 [=========================

3/3 [==============================] - 0s 2ms/step - loss: 0.1558 - auc_4: 0.9762
Epoch 5/50
3/3 [==============================] - 0s 2ms/step - loss: 0.1149 - auc_4: 0.9901
Epoch 6/50
3/3 [==============================] - 0s 2ms/step - loss: 0.1115 - auc_4: 0.9919
Epoch 7/50
3/3 [==============================] - 0s 2ms/step - loss: 0.1465 - auc_4: 0.9791
Epoch 8/50
3/3 [==============================] - 0s 2ms/step - loss: 0.1853 - auc_4: 0.9683
Epoch 9/50
3/3 [==============================] - 0s 2ms/step - loss: 0.2077 - auc_4: 0.9657
Epoch 10/50
3/3 [==============================] - 0s 2ms/step - loss: 0.1345 - auc_4: 0.9849
Epoch 11/50
3/3 [==============================] - 0s 2ms/step - loss: 0.1045 - auc_4: 0.9942
Epoch 12/50
3/3 [==============================] - 0s 2ms/step - loss: 0.1317 - auc_4: 0.9834
Epoch 13/50
3/3 [==============================] - 0s 2ms/step - loss: 0.1809 - auc_4: 0.9713
Epoch 14/50
3/3 [==============================] - 0s 2ms/step - loss: 0.1536

In [10]:
model_one_feature.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')


In [11]:
X_train, X_test, y_train, y_test = train_test_split(df_one_feature, labels, test_size=0.2, random_state=84)

h_new = model_one_feature.fit(
  X_train, # training data
  y_train, # training targets
  epochs=75,
  batch_size=50,
)


Epoch 1/75
4/4 [==============================] - 0s 3ms/step - loss: 5.2248 - accuracy: 0.6545
Epoch 2/75
4/4 [==============================] - 0s 1ms/step - loss: 1.2855 - accuracy: 0.4764
Epoch 3/75
4/4 [==============================] - 0s 1ms/step - loss: 0.8931 - accuracy: 0.5969
Epoch 4/75
4/4 [==============================] - 0s 1ms/step - loss: 0.7319 - accuracy: 0.6649
Epoch 5/75
4/4 [==============================] - 0s 1ms/step - loss: 0.6786 - accuracy: 0.7120
Epoch 6/75
4/4 [==============================] - 0s 1ms/step - loss: 0.5509 - accuracy: 0.7330
Epoch 7/75
4/4 [==============================] - 0s 1ms/step - loss: 0.5693 - accuracy: 0.7173
Epoch 8/75
4/4 [==============================] - 0s 1ms/step - loss: 0.5080 - accuracy: 0.7225
Epoch 9/75
4/4 [==============================] - 0s 1ms/step - loss: 0.5322 - accuracy: 0.6702
Epoch 10/75
4/4 [==============================] - 0s 1ms/step - loss: 0.4958 - accuracy: 0.7277
Epoch 11/75
4/4 [======================

In [12]:
y_pred = model_one_feature.predict(X_train)
rounded = [int(round(x[0])) for x in y_pred]
from sklearn.metrics import accuracy_score
a = accuracy_score(rounded,y_train)
print('Accuracy is:', a*100)

6/6 [==============================] - 0s 830us/step
Accuracy is: 85.34031413612566


In [13]:
y_pred = model_one_feature.predict(X_test)

rounded = [int(round(x[0])) for x in y_pred]
from sklearn.metrics import accuracy_score
a = accuracy_score(rounded,y_test)
print('Accuracy is:', a*100)

2/2 [==============================] - 0s 1ms/step
Accuracy is: 70.83333333333334


In [ ]:
from sklearn.inspection import permutation_importance
result = permutation_importance(model_one_feature, X_test, y_test, n_repeats=10, random_state=42, scoring='neg_mean_squared_error')

# print feature importances
fig, ax = plt.subplots()
col = ['0_x1', '1_x1','2_x1','3_x1', '4_x1','5_x1','6_x1']
ax.bar(col, result.importances_mean)
ax.set_xticklabels(col, rotation=90)
ax.set_title('Feature Importances from Permutation Importance')
plt.show()


1/2 [==============>...............] - ETA: 0s

In [ ]:
cm = confusion_matrix(rounded, y_test)
labels = ['Diseased', 'Healthy']

sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', xticklabels=labels, yticklabels=labels)

# Add labels and title
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')

plt.show()

In [ ]:
plt.plot(h_new.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

#Plot accuracy on same plt
#Downward trend for loss, and upward trend for the accuracy 